In [1]:
import torch
import torch.nn as nn
import scipy.stats as stats
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import optim
import matplotlib.pyplot as plt

In [2]:
from icnnet import ICNNet
from mydataset import MyDataset, get_gaussian_dataset, get_gaussian_transport_dataset
from toy_data_dataloader_gaussian import generate_gaussian_dataset, get_dataset, generate_dataset
from train_picnn import PICNNtrain
from train_wasserstein import train_wasserstein
from train_makkuva import train_makkuva, train_makkuva_epoch
from visualization import plot_transport

In [3]:
%load_ext autoreload
%autoreload 2

## __Generate dataset__


In [4]:
#dataset = get_dataset(d=2, r=100, N=500) #valou
#dataset = generate_gaussian_dataset(d=2, r=400, N=10000) #thomas
dataset = generate_dataset(d=2, r=500, N=100)
gaussian_dataset = get_gaussian_dataset(dataset)
gaussian_transport_dataset = get_gaussian_transport_dataset(gaussian_dataset)

c:\Users\dmgtr\OneDrive - Ecole Polytechnique\3A\P2\MAP588 - EA CondOT\CondOT\gaussian_transport.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u = torch.tensor(u)


## __Initialization__

### __PICNN training__

In [5]:
input_size = 2
layer_sizes = [input_size,64,64,64, 1]
n_layers = len(layer_sizes)

In [6]:
import torch.nn.functional as F

def get_embedding(C, c):
    scalar_product = torch.matmul(c.float(), C.t().float())
    embedding = F.softmax(scalar_product, dim=1)
    return(embedding)

context_layer_sizes = [2] * n_layers

In [7]:
model_init_f = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')
model_init_g = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')

In [20]:
print('training f')
gaussian_transport_dataloader = DataLoader(gaussian_transport_dataset, batch_size=250, shuffle=True)
PICNNtrain(model_init_f, gaussian_transport_dataloader, lr=0.001, epochs=150, init_z = lambda x: (1/2) * torch.norm(x, dim=-1, keepdim=True)**2)
#PICNNtrain(model_init_f, gaussian_transport_dataloader, lr=0.0001, epochs=1, init_z = lambda x: x)


# print('training g')
# reversed_gaussian_dataset = MyDataset(gaussian_dataset.Y, gaussian_dataset.C, gaussian_dataset.X)
# gaussian_transport_dataset_reversed = get_gaussian_transport_dataset(reversed_gaussian_dataset)
# gaussian_transport_dataloader_reversed = DataLoader(gaussian_transport_dataset_reversed, batch_size=250, shuffle=True)
# #PICNNtrain(model_init_g, gaussian_transport_dataloader_reversed, lr=0.0001, epochs=25, init_z = lambda x: (1/2) * torch.norm(-x, dim=-1, keepdim=True)**2)
# PICNNtrain(model_init_g, gaussian_transport_dataloader_reversed, lr=0.001, epochs=150, init_z = lambda x: (1/2) * torch.norm(x, dim=-1, keepdim=True)**2)

training f


c:\Users\dmgtr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([100, 1000, 2])) that is different to the input size (torch.Size([100, 1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/150 Loss: 0.00930789951235056
Epoch 2/150 Loss: 0.020446209236979485
Epoch 3/150 Loss: 0.015238824300467968
Epoch 4/150 Loss: 0.016683008521795273
Epoch 5/150 Loss: 0.008686402812600136
Epoch 6/150 Loss: 0.012463388964533806
Epoch 7/150 Loss: 0.013700975105166435
Epoch 8/150 Loss: 0.009687828831374645
Epoch 9/150 Loss: 0.008809277787804604
Epoch 10/150 Loss: 0.011523737572133541
Epoch 11/150 Loss: 0.010905868373811245
Epoch 12/150 Loss: 0.008489670231938362
Epoch 13/150 Loss: 0.00861630029976368
Epoch 14/150 Loss: 0.009903093799948692
Epoch 15/150 Loss: 0.009603611193597317
Epoch 16/150 Loss: 0.008255144581198692
Epoch 17/150 Loss: 0.008122265338897705
Epoch 18/150 Loss: 0.008986946195363998
Epoch 19/150 Loss: 0.008835595101118088
Epoch 20/150 Loss: 0.007945575751364231
Epoch 21/150 Loss: 0.007790749426931143
Epoch 22/150 Loss: 0.00825176015496254
Epoch 23/150 Loss: 0.008264499716460705
Epoch 24/150 Loss: 0.007741152308881283
Epoch 25/150 Loss: 0.007537400349974632
Epoch 26/150

In [21]:
state_dict_init_f = model_init_f.state_dict()
state_dict_init_g = model_init_g.state_dict()

In [22]:
# test=85
# n_points = 1000
# plot_transport(dataset, test, model_init_f, model_init_g, n_points=n_points)

In [23]:
# print('training f')
# gaussian_transport_dataloader = DataLoader(gaussian_transport_dataset, batch_size=250, shuffle=True)
# train_wasserstein(model_init_f, gaussian_transport_dataloader, lr=0.1, epochs=10, init_z = lambda x: (1/2) * torch.norm(x, dim=-1, keepdim=True)**2)

In [24]:
# X, Y, C = gaussian_dataset.X, gaussian_dataset.Y, gaussian_dataset.C
# #Calcul de la dérivée du PICNN

# for test in range(20):
#     x_i = X[test, :, :]
#     y_i = Y[test, :, :]
#     c_i = C[test, :, :]

#     locs = c_i[:,0]
#     #print(locs)

#     scales = c_i[:,1]
#     #print(scales)  


#     y_i.requires_grad_(True)
#     x_i.requires_grad_(True)
#     #c_i.requires_grad_(True)    

#     output_model_f = model_init_f(x_i, c_i)
#     grad_model_f = torch.autograd.grad(outputs=output_model_f, inputs=x_i, grad_outputs=torch.ones_like(output_model_f), create_graph=True)[0].detach().numpy()

#     plt.hist(X[test, :, 0],  bins=15, label = 'X', density = True)
#     plt.hist(Y[test, :, 0],  bins=15, label = 'Y', density = True)
#     plt.hist(grad_model_f[:, 0],  bins=15, label = 'grad_model', density = True, alpha = 0.5)
#     # plt.hist(X_pred,  bins=15, label = 'X_pred', density = True, alpha = 0.5)
#     interval_x = np.linspace(-3, 3, 300)
#     interval_y = np.linspace(-3*scales[0] + locs[0], 3*scales[0] + locs[0], 300)

#     plt.plot(interval_x, stats.norm.pdf(interval_x, loc=0, scale=1), label = 'X_distrib', color = 'blue')
#     plt.plot(interval_y, stats.norm.pdf(interval_y, loc = locs[0], scale = scales[0]), label = 'Y_distrib', color = 'orange')

#     plt.legend()
#     plt.show()


#     output_model_g = model_init_g(y_i, c_i)
#     grad_model_g = torch.autograd.grad(outputs=output_model_g, inputs=y_i, grad_outputs=torch.ones_like(output_model_g), create_graph=True)[0].detach().numpy()
#     plt.hist(X[test, :, 0],  bins=15, label = 'X', density = True, color = 'red')
#     #plt.hist(Y[test, :, 0],  bins=15, label = 'Y', density = True, color = 'blue')
#     plt.hist(grad_model_g[:, 0],  bins=15, label = 'grad_model', density = True, alpha = 0.5)
#     # plt.hist(X_pred,  bins=15, label = 'X_pred', density = True, alpha = 0.5)
#     interval_x = np.linspace(-3, 3, 300)
#     interval_y = np.linspace(-3*scales[0] + locs[0], 3*scales[0] + locs[0], 300)

#     plt.plot(interval_x, stats.norm.pdf(interval_x, loc=0, scale=1), label = 'X_distrib', color = 'blue')
#     #plt.plot(interval_y, stats.norm.pdf(interval_y, loc = locs[0], scale = scales[0]), label = 'Y_distrib', color = 'orange')

#     plt.legend()
#     plt.show()

## __Makkuva__

In [25]:
# state_dict_init_f = torch.load('trained_models/training7/models/model_f_0.pth')
# state_dict_init_g = torch.load('trained_models/training7/models/model_g_0.pth')

In [26]:
ICNNf = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')
ICNNg = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')

old_ICNNf = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')
old_ICNNg = ICNNet(layer_sizes = layer_sizes, context_layer_sizes=context_layer_sizes, init_bunne = 'TR')

# Load the state dictionary into ICNNf and ICNNg
ICNNf.load_state_dict(state_dict_init_f)
ICNNg.load_state_dict(state_dict_init_g)

old_ICNNf.load_state_dict(state_dict_init_f)
old_ICNNg.load_state_dict(state_dict_init_g)

l_ICNNf = [ICNNf, old_ICNNf]
l_ICNNg = [ICNNg, old_ICNNg]

In [30]:
n_points = 1000
test = 1

filepath_pth_f = 'trained_models/training7/models/model_f_'
filepath_pth_g = 'trained_models/training7/models/model_g_'

filepath_plt_f = 'trained_models/training7/plots/model_f_'
filepath_plt_g = 'trained_models/training7/plots/model_g_'

In [31]:
filename_pth_f = filepath_pth_f + str(0) + '.pth'
filename_pth_g = filepath_pth_g + str(0) + '.pth'
torch.save(ICNNf.state_dict(), filename_pth_f)
torch.save(ICNNg.state_dict(), filename_pth_g)

In [39]:
filename_plt_f = filepath_plt_f + str(0) + '.png'
filename_plt_g = filepath_plt_g + str(0) + '.png'
plot_transport(dataset, test, ICNNf, ICNNg, filename_f = filename_plt_f, filename_g = filename_plt_g)

<Figure size 1280x960 with 0 Axes>

In [37]:
dataloader = DataLoader(dataset, batch_size=500, shuffle=True)

loss_f = list()
loss_g = list()

for epoch in range(251, 351) :
    print('epoch :', epoch, end=('\r'))
    mean_loss_f, mean_loss_g = train_makkuva_epoch(l_ICNNf[epoch%2], l_ICNNg[epoch%2], l_ICNNf[1 - epoch%2], l_ICNNg[1 - epoch%2], dataloader, init_z_f = lambda x: (1/2) * torch.norm(x, dim=-1, keepdim=True)**2, init_z_g = lambda x: (1/2) * torch.norm(x, dim=-1, keepdim=True)**2, lr=0.001, train_freq_g=10, train_freq_f=1, gaussian_transport=False, regularize_f = False, regularize_g = True)
    #mean_loss_f, mean_loss_g = train_makkuva_epoch(ICNNf, ICNNg, None, None, dataloader, init_z_f = lambda x: (1/2) * torch.norm(x, dim=-1, keepdim=True)**2, init_z_g = lambda x: (1/2) * torch.norm(-x, dim=-1, keepdim=True)**2, lr=0.0001, train_freq_g=10, train_freq_f=1, gaussian_transport=False)

    loss_f.append(mean_loss_f)
    loss_g.append(mean_loss_g)

    filename_pth_f = filepath_pth_f + str(epoch) + '.pth'
    filename_pth_g = filepath_pth_g + str(epoch) + '.pth'
    torch.save(l_ICNNf[epoch%2].state_dict(), filename_pth_f)
    torch.save(l_ICNNg[epoch%2].state_dict(), filename_pth_g)

    filename_plt_f = filepath_plt_f + str(epoch) + '.png'
    filename_plt_g = filepath_plt_g + str(epoch) + '.png'
    plot_transport(dataset, test, l_ICNNf[epoch%2], l_ICNNg[epoch%2], filename_f = filename_plt_f, filename_g = filename_plt_g)

train_freq_g 10
train_freq_f 1
loss_g: 0.8513221740722656, loss_f: -0.14827898144721985
train_freq_g 10
train_freq_f 1
loss_g: 0.8790445327758789, loss_f: 0.005424084607511759
train_freq_g 10
train_freq_f 1
loss_g: 0.9345914125442505, loss_f: 0.045086801052093506
train_freq_g 10
train_freq_f 1
loss_g: 0.8980900049209595, loss_f: 0.04262238368391991
train_freq_g 10
train_freq_f 1
loss_g: 0.9109423756599426, loss_f: 0.03842322900891304
train_freq_g 10
train_freq_f 1
loss_g: 0.7769259214401245, loss_f: -0.1907612681388855
train_freq_g 10
train_freq_f 1
loss_g: 0.7901700139045715, loss_f: -0.2497549206018448
train_freq_g 10
train_freq_f 1
loss_g: 0.9639826416969299, loss_f: 0.06952670216560364
train_freq_g 10
train_freq_f 1
loss_g: 0.9895268678665161, loss_f: 0.19078482687473297
train_freq_g 10
train_freq_f 1
loss_g: 0.8184962272644043, loss_f: -0.21198277175426483
train_freq_g 10
train_freq_f 1
loss_g: 0.8220099210739136, loss_f: -0.09197838604450226
train_freq_g 10
train_freq_f 1
loss_g:

<Figure size 1280x960 with 0 Axes>

In [38]:
for ele in loss_f:
    print(ele)

print('stop')

for ele in loss_g:
    print(ele)

-0.14827898144721985
0.005424084607511759
0.045086801052093506
0.04262238368391991
0.03842322900891304
-0.1907612681388855
-0.2497549206018448
0.06952670216560364
0.19078482687473297
-0.21198277175426483
-0.09197838604450226
0.12439444661140442
0.09850722551345825
-0.11259204149246216
-0.15632280707359314
0.04727530851960182
0.03961414843797684
0.03605497628450394
-0.06310093402862549
-0.1443682461977005
0.051105812191963196
0.08225013315677643
-0.004855766426771879
-0.2657952606678009
0.030947290360927582
0.20716895163059235
-0.10214466601610184
-0.04814242571592331
0.06228938326239586
-0.01921173371374607
-0.03416843339800835
0.00836229044944048
0.09536775201559067
0.010086080059409142
-0.14750023186206818
-0.03132428973913193
0.042596034705638885
0.12270325422286987
-0.02795254811644554
-0.12717106938362122
0.1016254872083664
0.018921855837106705
-0.11902669072151184
0.07365439087152481
-0.006904115434736013
-0.2669472396373749
0.09713462740182877
0.21855615079402924
-0.200296759605

In [ ]:
ICNNf.load_state_dict(torch.load(filepath_pth_f + '0.pth'))
ICNNg.load_state_dict(torch.load(filepath_pth_g + '0.pth'))

filename_plt_f = 'trained_models/training7/plots/model_f_test'
filename_plt_g = 'trained_models/training7/plots/model_g_test'

plot_transport(dataset, test, ICNNf, ICNNg, filename_f = filename_plt_f, filename_g = filename_plt_g)